# Contents
4. [Build Opinions](#4.-Build-Opinions)  
    4.1. [Merge years](#4.1.-Merge-years)   
    4.2. [Preprocessing](#4.2.-Preprocessing)  
    ㅤㅤ4.2.1. [politicians -> quotes](#4.2.1.-politicians-->-quotes)  
    ㅤㅤ4.2.2. [quotes -> (topic, sentiment)](#4.2.2.-quotes-->-(topic,-sentiment))  
    4.3. [Build opinions](#4.3.-Build-opinions)

# **4. Build Opinions**

For each quote in the filtered dataset we have now extracted a topic and a sentiment, as showed in the previous notebooks. The next step is to combine this information in order to get, for each US (modern) politician, an aggregated opinion that reflects his views on all the topics. 

In this context, we define an **opinion** as a vector with dimension equal to the number of extracted topics, and values corresponding to the average sentiment for each topic. In parallel to that, we are also interested in how much each politician care about each topic, and we will capture that with a **count** vector whose entries correspond, for each politician, to the number of quotes per topic.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import gc
import natsort

In [ ]:
preprocess_folder = '/content/drive/MyDrive/ADA/Processed/'
topics_folder = '/content/drive/MyDrive/ADA/Topics/'
sentiment_folder = '/content/drive/MyDrive/ADA/Sentiment/'
opinion_folder = '/content/drive/MyDrive/ADA/Opinions/'
politicians_folder = '/content/drive/MyDrive/ADA/Politicians/'

In [ ]:
import datetime
import pytz
def printts(*objects):
    print(datetime.datetime.now(pytz.timezone('Europe/Zurich')).strftime("%d %b %Y %H:%M:%S"), ":", *objects)

## 4.1. Merge years

In [ ]:
def merge_df():
  '''
  Loads the preprocessed DataFrame for each year from 2015 to 2020 and merge them
  in a unique DataFrame.
  '''

  # Create list of preprocessed DataFrames per year
  df_years = []
  for filename in sorted(os.listdir(preprocess_folder), reverse=True):
    processpath = os.path.join(preprocess_folder, filename)
    printts(f'Reading {filename}...')
    df_year = pd.read_parquet(processpath)
    df_years.append(df_year)

  # Concatenate the processed years into one single dataframe
  printts(f'Combining years...')
  df = pd.concat(df_years)
  del df_year
  del df_years

  # Shuffle dataframe
  df = df.sample(frac=1, random_state=42)

  # Set index
  index = np.array(list(map(lambda x: 'q' + x, np.arange(len(df)).astype(str))))
  df = df.set_index(index)

  printts('Merging done')
  return df

## 4.2. Preprocessing

To create the **opinions** and **counts** matrix (PxT, where P = #politicians and T = #topics), we firstly need to create two supplementary DataFrames:
1. politicians -> quotes, allowing easy access to all the quotes by each politician.
2. quote -> (topic, sentiment), associating to each quote the topic and the sentiment previously extracted.

### 4.2.1. politicians -> quotes

In [ ]:
# Combine years
df = merge_df()

15 Dec 2021 14:55:36 : Reading quotes-2020.parquet.gzip...
15 Dec 2021 14:55:41 : Reading quotes-2019.parquet.gzip...
15 Dec 2021 14:55:58 : Reading quotes-2018.parquet.gzip...
15 Dec 2021 14:56:24 : Reading quotes-2017.parquet.gzip...
15 Dec 2021 14:56:53 : Reading quotes-2016.parquet.gzip...
15 Dec 2021 14:57:04 : Reading quotes-2015.parquet.gzip...
15 Dec 2021 14:57:16 : Combining years...
15 Dec 2021 14:58:02 : Merging done


In [ ]:
# Drop unneeded columns
df = df.drop(columns=['date', 'urls', 'domains'])

In [ ]:
# Map politicians to their quotes
df_politicians = df.groupby('qids').agg({'quotation' : lambda x : list(x.index),
                                         'speaker' : lambda x: x.iloc[0]})
del df

In [ ]:
# Rename column
df_politicians = df_politicians.rename(columns={'quotation' : 'indexes'})

df_politicians

,indexes,speaker
qids,,
Q1001235,"[q1717939, q2409021, q7050308]",Buddy Leach
Q1018322,[q1056399],Donald Lukens
Q1026119,"[q45368, q135526, q154606, q277076, q301235, q...",Cal Dooley
Q1027026,"[q11589, q20162, q20701, q35220, q40203, q4662...",Jeff Duncan
Q1027431,"[q108382, q118050, q132738, q180363, q307047, ...",Tim Wirth
...,...,...
Q984376,"[q445933, q1849204, q2055956, q3421575, q49316...",Malcolm Wallop
Q984509,"[q3538, q8435, q16651, q18091, q18902, q22752,...",John Sarbanes
Q989831,"[q481024, q921224, q921747, q942805, q1111455,...",Enrique Peñalosa


In [ ]:
# Save file
df_politicians.to_parquet(os.path.join(politicians_folder, 'politicians_raw.parquet'))

### 4.2.2. quotes -> (topic, sentiment)

In [ ]:
# Load topics
topics = pd.read_parquet(os.path.join(topics_folder, 'topics.parquet'))

In [ ]:
# Load sentiments and rename column
sentiments = pd.read_parquet(os.path.join(sentiment_folder, 'df_politicians_sentiment_only.parquet.gzip'))
sentiments = sentiments.rename(columns={'vader_compound_score': 'sentiment'})

Since both the topics and sentiments were assigned by chunks, appending then to parquet with pyarrow, we need to firstly sort the index to have them in the same order. We will use for this the quote index that we assigned in merging the years, performing a natural sort.

In [ ]:
# Natural sort of index for topics
index_sorted = natsort.natsorted(topics.index)
topics = topics.loc[index_sorted]

# Natural sort of index for sentiments
index_sorted = natsort.natsorted(sentiments.index)
sentiments = sentiments.loc[index_sorted]

We can now merge the two to create the target matrix.

In [ ]:
# merge topics and sentiments
df_quotes = topics.merge(sentiments, how='inner', left_index=True, right_index=True)

In [ ]:
df_quotes.head(10)

,topic,sentiment
q0,-1,-0.4019
q1,-1,-0.0572
q2,-1,0.7506
q3,-1,-0.3818
q4,443,0.0000
q5,-1,0.5574
q6,-1,0.8271
q7,629,0.4939
q8,6,0.4939
q9,-1,0.0000


## 4.3. Build opinions

We are now ready to create the opinions and counts matrices, defined as already discussed.

In [ ]:
NUM_TOPICS = 1000

In [ ]:
def convert_politicians_row(single_politicians_quotes, quotes_df, topics_count):
  """
  Build opinion and counts vector for a single politician
  """
  # initialize a list of sentiments and counts per topic
  topics_sentiments = {k: None for k in range(-1, topics_count)}
  topics_counts = {k: None for k in range(-1, topics_count)}

  # loop over quotes, adding sentiment and count to the lists
  for quote in single_politicians_quotes:
    try:
      topic = quotes_df.loc[quote]['topic']
      sentiment = quotes_df.loc[quote]['sentiment']
    except:
      print(quote)
      continue
    if topics_sentiments[topic] is not None:
      topics_sentiments[topic] += sentiment
      topics_counts[topic] += 1
    else:
      topics_sentiments[topic] = sentiment
      topics_counts[topic] = 1
    
  # average sentiments
  for topic in range(-1, topics_count):
    if topics_sentiments[topic] is not None:
      topics_sentiments[topic] /= topics_counts[topic]
  
  return pd.Series(topics_counts), pd.Series(topics_sentiments)

In [ ]:
def build_opinions(quotes_df, politicians_df):
  """
  Build opinions and counts matrices, including all politicians.
  """
  M = politicians_df.apply(lambda row: convert_politicians_row(row[0], quotes_df, NUM_TOPICS), axis=1)
  counts = pd.DataFrame(list(zip(*M))[0], index=M.index)
  opinions = pd.DataFrame(list(zip(*M))[1], index=M.index)
  return counts, opinions

In [ ]:
counts, opinions = build_opinions(df_quotes, df_politicians)

q9458184


In [ ]:
counts

,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
qids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Q1001235,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1018322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1026119,163.0,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1027026,1110.0,9.0,17.0,18.0,10.0,2.0,13.0,3.0,NaN,4.0,1.0,4.0,5.0,1.0,NaN,13.0,5.0,2.0,2.0,11.0,16.0,4.0,2.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,NaN,NaN,2.0,1.0,8.0,NaN,1.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1027431,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q984376,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q984509,477.0,7.0,8.0,NaN,2.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,3.0,NaN,5.0,6.0,1.0,9.0,NaN,2.0,NaN,NaN,1.0,3.0,4.0,2.0,3.0,NaN,2.0,NaN,NaN,1.0,NaN,5.0,2.0,NaN,2.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q989831,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
opinions

,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
qids,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Q1001235,0.724100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1018322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1026119,0.376801,NaN,0.000000,NaN,NaN,NaN,NaN,-0.632600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.65970,NaN,NaN,NaN,NaN,0.0026,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1027026,0.165928,-0.236622,0.063388,0.245711,0.25334,0.000000,-0.296454,0.451667,NaN,-0.09785,-0.47670,0.21105,0.319060,0.2263,NaN,0.038592,0.2308,0.208600,0.57725,0.14650,0.281481,0.4719,0.4783,0.3612,-0.440400,-0.1501,0.363000,0.2342,0.072033,0.5106,NaN,NaN,0.0386,-0.24840,0.12885,NaN,0.0000,0.3382,0.5267,-0.02135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.47678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.373567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01155,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q1027431,0.057248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,-0.15930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.106475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15310,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q984376,0.131317,0.883400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.177900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q984509,0.189572,0.204371,0.059887,NaN,-0.14800,0.091733,0.481225,0.361200,0.148,0.29375,-0.07655,0.03960,-0.098667,NaN,0.23686,0.217733,0.0000,0.042911,NaN,-0.19255,NaN,NaN,0.6486,-0.1187,-0.419075,0.0386,-0.083800,NaN,0.630400,NaN,NaN,0.0,NaN,0.31398,0.00000,NaN,0.1145,NaN,-0.1877,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q989831,0.055178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Convert columns types
opinions.columns = opinions.columns.astype(str)
counts.columns = counts.columns.astype(str)

In [ ]:
# Save matrices
opinions.to_parquet(os.path.join(opinion_folder, 'opinions.parquet'))
counts.to_parquet(os.path.join(opinion_folder, 'counts.parquet'))